In [6]:
from pycaret.classification import *
import pandas as pd

In [7]:
import os
os.getcwd()

'/Users/parmidachoubsaz/ML_Final_Project_Abalone'

In [8]:
os.listdir()

['abalone_sex_classification.ipynb',
 'logs.log',
 'abalone.names',
 '.ipynb_checkpoints',
 'abalone.data']

In [9]:
df = pd.read_csv("abalone.data", header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [53]:
df.columns = [
    'Sex','Length','Diameter','Height',
    'WholeWeight','ShuckedWeight','VisceraWeight','ShellWeight','Rings'
]

df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [54]:
df.shape

(4177, 9)

In [55]:
df['Sex'].value_counts()

Sex
M    1528
I    1342
F    1307
Name: count, dtype: int64

In [57]:
df.describe()

,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [11]:
from pycaret.classification import *

clf = setup(
    data = df,
    target = 'Sex',
    session_id = 123,
    normalize = True
)

,Description,Value
0,Session id,123
1,Target,Sex
2,Target type,Multiclass
3,Target mapping,"F: 0, I: 1, M: 2"
4,Original data shape,"(4177, 9)"
5,Transformed data shape,"(4177, 9)"
6,Transformed train set shape,"(2923, 9)"
7,Transformed test set shape,"(1254, 9)"
8,Numeric features,8
9,Preprocess,True


In [12]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.5617,0.0000,0.5617,0.5479,0.5473,0.3397,0.3441,0.2550
ridge,Ridge Classifier,0.5570,0.0000,0.5570,0.5437,0.5409,0.3342,0.3398,0.0070
lda,Linear Discriminant Analysis,0.5484,0.0000,0.5484,0.5412,0.5394,0.3186,0.3216,0.0070
gbc,Gradient Boosting Classifier,0.5474,0.0000,0.5474,0.5361,0.5368,0.3167,0.3196,0.2020
rf,Random Forest Classifier,0.5443,0.7408,0.5443,0.5376,0.5395,0.3140,0.3148,0.0650
et,Extra Trees Classifier,0.5378,0.7383,0.5378,0.5299,0.5322,0.3044,0.3054,0.0470
lightgbm,Light Gradient Boosting Machine,0.5375,0.7373,0.5375,0.5316,0.5332,0.3036,0.3044,0.6380
svm,SVM - Linear Kernel,0.5340,0.0000,0.5340,0.5190,0.5001,0.3012,0.3163,0.0110
ada,Ada Boost Classifier,0.5316,0.0000,0.5316,0.5142,0.5086,0.2920,0.2995,0.0280
qda,Quadratic Discriminant Analysis,0.5265,0.0000,0.5265,0.5136,0.4942,0.2861,0.2990,0.0070


In [15]:
final_model = finalize_model(best_model)

In [24]:
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [17]:
save_model(final_model, "abalone_sex_classifier")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Length', 'Diameter', 'Height',
                                              'WholeWeight', 'ShuckedWeight',
                                              'VisceraWeight', 'ShellWeight',
                                              'Rings'],
                                     transformer=SimpleImputer(add_indicator=Fa...
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=StandardScaler(copy=True,
                                                                with_mean=True,
                                                                with_std=True))),
     

In [18]:
predictions = predict_model(final_model)
predictions.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5463,0.7602,0.5463,0.5316,0.5303,0.3157,0.3204


,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings,Sex,prediction_label,prediction_score
1948,0.635,0.515,0.165,1.2290,0.5055,0.2975,0.3535,10,M,F,0.5433
956,0.495,0.400,0.135,0.6100,0.2720,0.1435,0.1440,7,M,I,0.4179
1341,0.585,0.490,0.185,1.1710,0.5220,0.2535,0.3350,10,M,M,0.4927
821,0.370,0.275,0.140,0.2215,0.0970,0.0455,0.0615,6,I,I,0.7898
569,0.410,0.320,0.115,0.3870,0.1650,0.1005,0.0985,11,F,M,0.3916


In [19]:
results_compare = pull()
results_compare

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5463,0.7602,0.5463,0.5316,0.5303,0.3157,0.3204


In [22]:
results_compare.to_csv("model_results.csv", index=False)

In [23]:
predictions.to_csv("predictions.csv", index=False)

In [48]:
plot_model(final_model, plot='confusion_matrix', save=True)

'Confusion Matrix.png'

In [38]:
plot_model(final_model, plot='class_report', save=True)

'Class Report.png'

In [33]:
plot_model(final_model, plot='feature', save=True)


'Feature Importance.png'

In [34]:
plot_model(final_model, plot='learning', save=True)

'Learning Curve.png'

In [49]:
plot_model(final_model, plot='pr', save=True)

'Precision Recall.png'

In [50]:
plot_model(final_model, plot='error', save=True)

'Prediction Error.png'

In [51]:
import os
os.makedirs("plots", exist_ok=True)

for file in ["Class Report.png", "Confusion Matrix.png",
             "Feature Importance.png", "Learning Curve.png",
             "Precision Recall.png", "Prediction Error.png"]:
    os.rename(file, f"plots/{file}")

In [58]:
results_compare = pull()
results_compare

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5463,0.7602,0.5463,0.5316,0.5303,0.3157,0.3204


In [59]:
results_compare.to_csv("model_results.csv", index=False)

Abalone Sex Classification – Summary

Dataset: 
UCI Abalone (4177 samples, 9 features)

Task: 
Multiclass Classification (predicting Sex: M / F / I)

Method: 
PyCaret Classification Module

Best Model: 
Logistic Regression

Cross-Validation Performance:
- Accuracy: ~0.55
- AUC: ~0.76

Saved Outputs:
- abalone_sex_classifier.pkl – final model
- model_results.csv – model comparison table
- predictions.csv – predictions on test set

Plots saved as PNG:
- Confusion Matrix
- Classification Report
- Feature Importance
- Learning Curve
- Precision–Recall Curve
- Prediction Error